Notebook that won't work because.

In [1]:
from smi2wyk.ccsd_utils import search_ccdc_for_organic_mols

all_organic_mols = search_ccdc_for_organic_mols(-1)

In [2]:
from tqdm import tqdm


In [3]:
smiles_list = [hit.molecule.smiles for hit in tqdm(all_organic_mols)]

 10%|▉         | 36736/379449 [01:57<18:18, 312.07it/s]

KeyboardInterrupt: 

 10%|▉         | 36736/379449 [02:10<18:18, 312.07it/s]

Let's convert these smiles into rdkit molecules objects

In [ ]:
import pandas as pd

from rdkit import Chem
from rdkit.Chem.Descriptors import MolWt

In [ ]:
def convert_smiles_to_mol(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol
    except Exception as e:
        print(f'Exception {e} for smiles: {smiles}')
        return None


df_csd = pd.DataFrame(smiles_list, columns=['smiles'])
tqdm.pandas()

df_csd['mol'] = df_csd['smiles'].progress_apply(convert_smiles_to_mol)
# print length before and after dropping nans
print(f'Length before dropping invalid mols: {len(df_csd)}')
print(f'Length after dropping invalid mols {len(df_csd.dropna())}')

df_csd = df_csd.dropna()
df_csd['MW'] = df_csd['mol'].progress_apply(MolWt)

[13:28:39] Explicit valence for atom # 11 O, 4, is greater than permitted
[13:28:39] Explicit valence for atom # 5 Ca, 3, is greater than permitted
[13:28:39] Explicit valence for atom # 11 O, 3, is greater than permitted
[13:28:39] Explicit valence for atom # 8 C, 6, is greater than permitted
[13:28:39] Explicit valence for atom # 19 Li, 3, is greater than permitted
[13:28:39] Explicit valence for atom # 16 N, 4, is greater than permitted
[13:28:39] Explicit valence for atom # 9 B, 4, is greater than permitted
[13:28:39] Explicit valence for atom # 1 B, 4, is greater than permitted
[13:28:39] Explicit valence for atom # 2 O, 3, is greater than permitted
[13:28:39] Explicit valence for atom # 2 O, 3, is greater than permitted
[13:28:39] Explicit valence for atom # 3 O, 3, is greater than permitted
[13:28:39] Explicit valence for atom # 14 N, 4, is greater than permitted
[13:28:39] Explicit valence for atom # 8 N, 4, is greater than permitted
[13:28:39] Explicit valence for atom # 9 N, 

Exception No registered converter was able to produce a C++ rvalue of type std::__1::basic_string<wchar_t, std::__1::char_traits<wchar_t>, std::__1::allocator<wchar_t> > from this Python object of type NoneType for smiles: None
Exception No registered converter was able to produce a C++ rvalue of type std::__1::basic_string<wchar_t, std::__1::char_traits<wchar_t>, std::__1::allocator<wchar_t> > from this Python object of type NoneType for smiles: None
Exception No registered converter was able to produce a C++ rvalue of type std::__1::basic_string<wchar_t, std::__1::char_traits<wchar_t>, std::__1::allocator<wchar_t> > from this Python object of type NoneType for smiles: None


[13:28:39] Explicit valence for atom # 7 C, 6, is greater than permitted
[13:28:39] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22
[13:28:39] Explicit valence for atom # 12 O, 3, is greater than permitted
[13:28:39] Explicit valence for atom # 37 K, 7, is greater than permitted
100%|██████████| 1000/1000 [00:00<00:00, 2623.72it/s]


Length before dropping invalid mols: 1000
Length after dropping invalid mols 964


100%|██████████| 964/964 [00:00<00:00, 198075.20it/s]


In [ ]:
from rdkit.Chem.MolStandardize import rdMolStandardize

largest_Fragment = rdMolStandardize.LargestFragmentChooser()

def largest_mass(mol):
    # largest_mass = 0
    # for sub_mol in mol.GetFragments():
    #     mass = MolWt(sub_mol)
    #     if mass > largest_mass:
    #         largest_mass = mass
    
    largest_mol = largest_Fragment.choose(mol)
    largest_mass = MolWt(largest_mol)
    return largest_mass
df_csd['MW_largest'] = df_csd['mol'].progress_apply(largest_mass)

100%|██████████| 964/964 [00:00<00:00, 2249.21it/s]


In [ ]:
import plotly.express as px

px.histogram(df_csd, x='MW_largest')

/Users/williammccorkindale/miniconda3/envs/csd_env/lib/python3.7/site-packages/plotly/io/_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/williammccorkindale/miniconda3/envs/csd_env/lib/python3.7/site-packages/plotly/io/_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [ ]:
lets_keep = df_csd.query('MW_largest > 100 & MW_largest < 700').sort_values(by='MW_largest', ascending=False)
print(f'length of dataframe we are keeping: {len(lets_keep)}')
lets_keep.smiles.values

length of dataframe we are keeping: 914


array(['Oc1ccc(Br)cc1C=NCCN(CCN=Cc1cc(Br)ccc1O)CCN=Cc1cc(Br)ccc1O',
       'Oc1ccc(Br)cc1C=NCCN(CCN=Cc1cc(Br)ccc1O)CCN=Cc1cc(Br)ccc1O',
       'Oc1ccc(Br)cc1C=NCCN(CCN=Cc1cc(Br)ccc1O)CCN=Cc1cc(Br)ccc1O',
       '[Te](c1ccccc1)c1nc([Te]c2ccccc2)nc([Te]c2ccccc2)n1',
       'OCCCOCC1=CSC(=N1)C1=NC(=C(S1)C1=C(COCCCO)N=C(S1)C1=NC(=CS1)COCCCO)COCCCO',
       'OCCCOCC1=CSC(=N1)C1=NC(=C(S1)C1=C(COCCCO)N=C(S1)C1=NC(=CS1)COCCCO)COCCCO',
       'Cc1cc(cc(c1P=C(c1cccc(n1)C(=Pc1c(C)cc(cc1C(C)(C)C)C(C)(C)C)C(C)(C)C)C(C)(C)C)C(C)(C)C)C(C)(C)C',
       'Cc1ccc(cc1)S(=O)(=O)N1C(=O)C(CC(F)(F)F)(c2ccccc2)C(=C(I)c2ccccc2)c2ccccc12',
       'COC(=O)C1=NN2C=C1C1(CCCCC1)O[Si](C(C)C)(C(C)C)N1C=C(C(=N1)C(=O)OC)C1(CCCCC1)O[Si]2(C(C)C)C(C)C.COC(=O)C1=NN2C=C1C1(CCCCC1)O[Si](C(C)C)(C(C)C)N1C=C(C(=N1)C(=O)OC)C1(CCCCC1)O[Si]2(C(C)C)C(C)C',
       'O[Si](c1ccccc1)(c1ccccc1)c1cc(cc(c1)[Si](O)(c1ccccc1)c1ccccc1)[Si](O)(c1ccccc1)c1ccccc1.O',
       'CC1C(CCC2C1(C)CCC1C2(C)CCC2(C)C3CC(C)(C)CCC3(C)C(CC12C)OC(=O)c1ccc(Br)c